# Preparing an AF state in the Ising model

This purpose of this notebook is to illustrate how to build a sequence for preparing an AF state in the Ising model. This notebook is supposed to evolve with code updates. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip

from pulser import Pulse, Sequence, Register
from pulser.waveforms import ConstantWaveform, RampWaveform
from pulser.devices import Chadoq2
from pulser.simulation import Simulation

## Initializing my device

In [ ]:
n_side = 5
coords = 13 * np.array([(x,y) for x in range(n_side) for y in range(n_side) 
                    if (x in {0,n_side-1} or y in {0,n_side-1})], dtype=np.float64)
reg = Register.from_coordinates(coords, prefix='atom')
reg.draw()
device = Chadoq2(reg) 

In [ ]:
reg = Register.square(side = 3, spacing=5, prefix='q')
reg.draw()
device = Chadoq2(reg) 

## Defining my waveforms 

We are realizing the following program

<img src="files/AF_Ising_program.png" alt="AF Pulse Sequence" style="width: 320px;"/>

In [ ]:
# Parameters in MHz and ns
#U = 1 * 2*np.pi  # btw 1-3 Mhz*2pi. Should we include this VdW magnitude in an AFM-type Simulation?

delta_0 = -2 * 2*np.pi
delta_f = 6 * 2*np.pi
Omega_max = 2.3 * 2*np.pi  # btw 1.8-2 Mhz*2pi
t_rise = 250
t_fall = 500
t_sweep = (delta_f-delta_0)/(2*np.pi*10) * 1000


In [ ]:
# Parameters in MHz and ns
#U = 1 * 2*np.pi  # btw 1-3 Mhz*2pi. Should we include this VdW magnitude in an AFM-type Simulation?

delta_0 = -.2 * 2*np.pi
delta_f = .4 * 2*np.pi
Omega_max = .23 * 2*np.pi  # btw 1.8-2 Mhz*2pi
t_rise = 25
t_fall = 50
t_sweep = (delta_f-delta_0)/(2*np.pi*10) * 100



In [ ]:
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0., Omega_max),
                              delta_0, 
                              0.)
sweep = Pulse.ConstantAmplitude(Omega_max,
                                RampWaveform(t_sweep, delta_0, delta_f),
                                0.)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.),
                              delta_f,
                              0.)

## Creating my sequence

In [ ]:
seq = Sequence(device)
seq.declare_channel('ising', 'rydberg_global')

seq.add(rise, 'ising')
seq.add(sweep, 'ising')
seq.add(fall, 'ising')

print(seq)
seq.draw()

## Phase Diagram

In [ ]:
#phase = {'omega':[], 'delta':[], 'time': range(max(seq._last(ch).tf for ch in seq.declared_channels))}
delta = []
omega = []
for x in seq._schedule['ising']:
    if isinstance(x.type,Pulse):
        omega += list(x.type.amplitude.samples)
        delta += list(x.type.detuning.samples)


In [ ]:
fig, ax = plt.subplots()
ax.plot(np.array(delta)/(2*np.pi),np.array(omega)/(2*np.pi),'red',lw=2)
ax.grid(True, which='both')
ax.set_xlabel(r"$\delta(t)/2\pi$")
ax.set_ylabel(r"$\Omega(t)/2\pi$")
ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')
ax.set_facecolor('0.9')

# Simulation

In [ ]:
simul = Simulation(seq)

In [ ]:
up = qutip.basis(2,0)
def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
occup_list = [occupation(j) for j in range(simul._size)]

By parity symmetry, we only need to observe half of the atoms, the other one reflects the results

In [ ]:
simul.run(obs_list=occup_list, progress_bar=True)

In [ ]:
for expv in simul.output.expect:
    plt.plot(expv)

In [ ]:
L = n_side
res = np.zeros((L,L))
pos = [k for k in range(L*L) if (k//L in {0,L-1} or k%L in {0,L-1})] # valid position indexes in atom array
for i,ev in enumerate(simul.output.expect):
    x = pos[i]//L
    y = pos[i]%L
    res[x,y] = ev[-1]
plt.matshow(res, cmap='hot')
print(res)

In [ ]:
L = int(np.sqrt(len(reg.qubits)))
res=np.zeros((L,L))
for i,ev in enumerate(simul.output.expect):
    res[i//L,i%L] = ev[-1]
plt.matshow(res, cmap='hot')
print(res)

In [ ]:
len(reg.qubits)